## 머신러닝 연습 : Titanic
### [Base Line Model] : 기본적인 로직 위주의 간단한 모델 
* 대일감정원 데이터분석 스터디 2024.02.01

* 타이타닉호 침몰사건
https://namu.wiki/w/%ED%83%80%EC%9D%B4%ED%83%80%EB%8B%89%ED%98%B8%20%EC%B9%A8%EB%AA%B0%20%EC%82%AC%EA%B3%A0

## 1.기본 라이브러리 불러오기

In [2]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

## 2. 데이터 불러오기

### 2.1 데이터 출처

* 데이터 출처 : kaggle
* Titanic - Machine Learning from Disaster
* https://www.kaggle.com/competitions/titanic

### 2.2 데이터 수집

####  csv 파일 다운 받는 방법

* Kaggle 회원가입 후, 접속해서 파일 다운 받기
* https://www.kaggle.com/competitions/titanic/data

* 3가지 파일을 다운받는다.
* train.csv, test.csv, gender_submmision.csv

* train.csv - the training set (생존여부 데이터 있음)
* test.csv - the test set (생존여부 데이터 없음)
* 위 두 파일은 타이타닉 탑승자의 정보와 침몰 후 생존여부를 train과 test로 나누어 놓은 파일임.
* gender_submission.csv : test.csv의 Passenger_id와 생존여부만 있는 파일 (캐글 제출용)

### 2.3 데이터 불러오기

In [3]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

## 3. 탐색적 데이터 분석 (EDA : Exploratory Data Analysis)

In [4]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [6]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [7]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [8]:
print('train data 크기 :', train.shape)
print('test data 크기 :', test.shape)

train data 크기 : (891, 12)
test data 크기 : (418, 11)


In [9]:
# test data에는 Survived 컬럼이 없음
# train data로 모델을 학습시켜 test data의 생존 여부를 예측해야 함

In [10]:
'''
[데이터 정보]

PassengerId - 탑승자 데이터 일련번호
Survived - 생존유무, target 값. (0 = 사망, 1 = 생존)
Name - 탑승객 성명
Pclass - 티켓 클래스. (1 = 1st, 2 = 2nd, 3 = 3rd)
Sex - 성별
Age - 나이
SibSp - 함께 탑승한 형제자매, 배우자 수 총합
Parch - 함께 탑승한 부모, 자녀 수 총합
Embarked - 탑승 항구
Fare - 탑승 요금
Ticket - 티켓 넘버
Cabin - 객실 넘버
'''

'\n[데이터 정보]\n\nPassengerId - 탑승자 데이터 일련번호\nSurvived - 생존유무, target 값. (0 = 사망, 1 = 생존)\nName - 탑승객 성명\nPclass - 티켓 클래스. (1 = 1st, 2 = 2nd, 3 = 3rd)\nSex - 성별\nAge - 나이\nSibSp - 함께 탑승한 형제자매, 배우자 수 총합\nParch - 함께 탑승한 부모, 자녀 수 총합\nEmbarked - 탑승 항구\nFare - 탑승 요금\nTicket - 티켓 넘버\nCabin - 객실 넘버\n'

In [11]:
submission = pd.read_csv('./data/gender_submission.csv')

In [12]:
submission.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [13]:
# test.csv 파일에서 PassengerId 칼럼만 가져왔고,
# Survived 칼럼은 임의의 값으로 채워져 있음.
# 여러분이 머신러닝 모델을 학습시킨 후, test.csv에 있는 승객들의 생존여부를 결정하여
# 이 파일의 Survived 칼럼에 정답을 채운 후, 제출하면 됨.

## 4. 데이터 전처리

### 1) Feature Engineering : feature selection

In [14]:
# 생존여부와 관련있는 컬럼만 선택
# 결측치가 많은 컬럼은 우선 제외하기로 함

In [15]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [16]:
# 생존여부와 무관한 칼럼 : PassengerId, Name, Ticket

In [17]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [18]:
# 결측치가 많은 칼럼 : Age, Cabin
# 데이터를 함부로 삭제하는 것은 안됨.
# 결측치 처리하는 방법 : 1.결측치 확인 > 2.처리방법 결정(삭제 or 살릴 것이냐) > 3.채운다면 어떻게 채울 것인지
# 여기서는 우선 삭제하기로 함.
# 채우는 방법은 찾아보기 바람. ('머신러닝 결측치 처리'로 검색)

In [19]:
train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [20]:
# PassengerId, Name, Ticket, Age, Cabin 칼럼을 제외한 새로운 데이터 프레임을 만듬
new_train = train[['Survived', 'Pclass', 'Sex', 'SibSp', 'Parch', 'Fare', 'Embarked']]

In [21]:
new_train.head()

,Survived,Pclass,Sex,SibSp,Parch,Fare,Embarked
0,0,3,male,1,0,7.2500,S
1,1,1,female,1,0,71.2833,C
2,1,3,female,0,0,7.9250,S
3,1,1,female,1,0,53.1000,S
4,0,3,male,0,0,8.0500,S


### 2) 결측치 처리

In [22]:
new_train.isnull().sum()

Survived    0
Pclass      0
Sex         0
SibSp       0
Parch       0
Fare        0
Embarked    2
dtype: int64

In [25]:
# Embaked에만 2개의 결측치가 있음.
# 몇 개 안되니 해당 행은 제외함.
# 채울것이면 어떻게 채울지는 '머신러닝 결측치 처리'로 찾아보기 바람.

In [24]:
new_train.dropna(inplace=True)

In [26]:
new_train.isnull().sum()

Survived    0
Pclass      0
Sex         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

### 3) Label Encoding

In [27]:
# 머신러닝은 문자열을 입력받지 못함
# 문자열을 숫자로 변환해줘야 함
# 숫자로 바꿔주는 방법은 대략 2가지임.
# 1. 라벨 인코딩 : 문자열을 숫자로 변환 (ex. 남자:1, 여자:2)
# 2. 원 핫 인코딩 : 문자열을 숫자로 변환하되, 칼럼을 새로 만들어서 변환 (man column, woman column을 새로 만들고, 남자면 man에 1, woman에 0을 넣는 식)
# https://cori.tistory.com/163

In [28]:
new_train.head()

,Survived,Pclass,Sex,SibSp,Parch,Fare,Embarked
0,0,3,male,1,0,7.2500,S
1,1,1,female,1,0,71.2833,C
2,1,3,female,0,0,7.9250,S
3,1,1,female,1,0,53.1000,S
4,0,3,male,0,0,8.0500,S


In [29]:
new_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 889 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  889 non-null    int64  
 1   Pclass    889 non-null    int64  
 2   Sex       889 non-null    object 
 3   SibSp     889 non-null    int64  
 4   Parch     889 non-null    int64  
 5   Fare      889 non-null    float64
 6   Embarked  889 non-null    object 
dtypes: float64(1), int64(4), object(2)
memory usage: 55.6+ KB


In [30]:
# 숫자가 아닌 칼럼은 sex와 embarked 두개임.

In [31]:
new_train['Sex'].value_counts()

male      577
female    312
Name: Sex, dtype: int64

In [32]:
# male과 female을 각각 0과 1로 변환

In [33]:
new_train['Sex'] = new_train['Sex'].replace({'male': 0, 'female': 1})

In [34]:
# 위에서는 replace를 사용했지만
# 보통은 LabelEncoder, OneHotEncoder를 사용함.
# get_dummies를 사용하면 one-hot encoding을 할 수 있음.

In [35]:
new_train.head()

,Survived,Pclass,Sex,SibSp,Parch,Fare,Embarked
0,0,3,0,1,0,7.2500,S
1,1,1,1,1,0,71.2833,C
2,1,3,1,0,0,7.9250,S
3,1,1,1,1,0,53.1000,S
4,0,3,0,0,0,8.0500,S


In [36]:
new_train['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [37]:
# S, C, Q를 각각 0, 1, 2로 변환
new_train['Embarked'] = new_train['Embarked'].replace({'S': 0, 'C': 1, 'Q': 2})

In [38]:
new_train.head()

,Survived,Pclass,Sex,SibSp,Parch,Fare,Embarked
0,0,3,0,1,0,7.2500,0
1,1,1,1,1,0,71.2833,1
2,1,3,1,0,0,7.9250,0
3,1,1,1,1,0,53.1000,0
4,0,3,0,0,0,8.0500,0


In [39]:
new_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 889 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  889 non-null    int64  
 1   Pclass    889 non-null    int64  
 2   Sex       889 non-null    int64  
 3   SibSp     889 non-null    int64  
 4   Parch     889 non-null    int64  
 5   Fare      889 non-null    float64
 6   Embarked  889 non-null    int64  
dtypes: float64(1), int64(6)
memory usage: 55.6 KB


In [41]:
# 이 부분 쯤에 이상치(outlier) 처리를 추가해야 함.
# 이상치 처리 방법은 여러가지가 있지만
# 순서는 : 1.이상치 확인 > 2.이상치 처리방법 결정(삭제 or 살릴 것이냐) > 3.살린다면 어떻게 할 것인지
# https://sseozytank.tistory.com/18


# 5. Model 학습 및 평가

### 1) target과 feature로 분리

In [42]:
y = new_train['Survived']
X = new_train.drop('Survived', axis=1)

In [43]:
X.head()

,Pclass,Sex,SibSp,Parch,Fare,Embarked
0,3,0,1,0,7.2500,0
1,1,1,1,0,71.2833,1
2,3,1,0,0,7.9250,0
3,1,1,1,0,53.1000,0
4,3,0,0,0,8.0500,0


In [44]:
y.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

### 2) 훈련 데이터와 평가 데이터로 분리

In [45]:
from sklearn.model_selection import train_test_split

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=2024)

In [47]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((711, 6), (178, 6), (711,), (178,))

### 3) 모델 선택 및 학습

In [50]:
# 1.분류 모델(Classifcation)이냐 회귀 모델(Regression)이냐를 먼저 파악해야 함.
# 2.여기서는 생존여부(1, 0)를 예측하는 것이므로 분류 모델을 사용해야 함.
# 3.알고리즘 선택은 자유이나, 최근에 일반적으로 많이 사용하는 모델은 랜덤포레스트, XGBoost, LightGBM 등이 있음.
# 4.여기서는 랜덤포레스트 분류(Classification) 모델을 사용하기로 함.
# 5.보통은 여러 모델을 사용해서 비교해보고 가장 성능이 좋은 모델을 선택함.
from sklearn.ensemble import RandomForestClassifier

In [51]:
rf_clf = RandomForestClassifier(random_state=2024)  # 객체 생성
# 하이퍼파라미터 튜닝을 하지 않았으므로 기본값으로 진행됨.
# 하이퍼 파라미터 튜닝을 하려면 여기서 파라미터를 설정해주면 됨.
# ex) RandomForestClassifier(n_estimators=100, max_depth=10, random_state=2024)
# 'RandomForestClassifier의 하이퍼파라미터' 튜닝으로 검색하면 됨.
# 자동튜딩 방법인 GridSearchCV, RandomizedSearchCV를 사용하면 더 편리하니 찾아보기 바람.

In [52]:
rf_clf.fit(X_train, y_train)                        # 학습 진행 (fit 메소드 사용)

RandomForestClassifier(random_state=2024)

### 4) 모델 평가

In [53]:
prediction = rf_clf.predict(X_test)           # X_test 의 생존여부 값 예측 (predict 메소드 사용)

In [54]:
prediction

array([1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0,
       1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0,
       0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 1], dtype=int64)

In [55]:
# 검증은 예측값(prediction)과 실제값(y_test)을 비교하여 진행함.
from sklearn.metrics import accuracy_score
# '분류 모델의 평가지표'로 검색하면 다른 평가지표도 찾아볼 수 있음.
# accuracy_score 외에도 precision, recall, f1_score, roc_auc_score 등.

In [57]:
rf_score = accuracy_score(y_test, prediction)
print('Random Forest 정확도 :', rf_score)

Random Forest 정확도 : 0.8033707865168539


In [58]:
# 정확도가 80%가 나왔음.

In [63]:
# 이 학습한 모델을 저장하고 싶다면
# import joblib
# joblib.dump(rf_clf, 'random_forest_model.pkl')
# 저장한 모델을 불러오고 싶다면
# loaded_model = joblib.load('random_forest_model.pkl')

# 6. Test 데이터에 대한 예측

### 1) 데이터 전처리

In [64]:
# 예측대상 데이터도 train 데이터와 같은 형태로 전처리를 해줘야 함.
# 보통은 위에서 train 데이터 전처리 할때 같이 하는 경우가 많음.

In [65]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [66]:
new_train.head()

,Survived,Pclass,Sex,SibSp,Parch,Fare,Embarked
0,0,3,0,1,0,7.2500,0
1,1,1,1,1,0,71.2833,1
2,1,3,1,0,0,7.9250,0
3,1,1,1,1,0,53.1000,0
4,0,3,0,0,0,8.0500,0


In [67]:
test.columns

Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [68]:
new_test = test[['Pclass', 'Sex', 'SibSp', 'Parch', 'Fare', 'Embarked']]

In [69]:
new_test.head()

,Pclass,Sex,SibSp,Parch,Fare,Embarked
0,3,male,0,0,7.8292,Q
1,3,female,1,0,7.0000,S
2,2,male,0,0,9.6875,Q
3,3,male,0,0,8.6625,S
4,3,female,1,1,12.2875,S


In [70]:
new_test['Sex'] = new_test['Sex'].replace({'male': 0, 'female': 1})
new_test['Embarked'] = new_test['Embarked'].replace({'S': 0, 'C': 1, 'Q': 2})

In [71]:
new_test.head()

,Pclass,Sex,SibSp,Parch,Fare,Embarked
0,3,0,0,0,7.8292,2
1,3,1,1,0,7.0000,0
2,2,0,0,0,9.6875,2
3,3,0,0,0,8.6625,0
4,3,1,1,1,12.2875,0


In [72]:
new_test.isnull().sum()

Pclass      0
Sex         0
SibSp       0
Parch       0
Fare        1
Embarked    0
dtype: int64

In [73]:
# fare에 결측치가 하나 있음, 
# kaggle 문제에서 모든 테스트 데이터에 대해 예측하라고 했으므로 결측치를 삭제할 수는 없음.
# 결측값을 해당 칼럼의 평균값으로 대체함.
new_test['Fare'].fillna(new_test['Fare'].mean(), inplace=True)

In [74]:
new_test.isnull().sum()

Pclass      0
Sex         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

### 2) 예측값 생성

In [75]:
test_prediction = rf_clf.predict(new_test)      

In [76]:
test_prediction

array([0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0,
       1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1,
       1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1,
       0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

### 3) 제출하기

In [78]:
# 제출할 파일 불러오기
submission = pd.read_csv('./data/gender_submission.csv')

In [79]:
submission.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [81]:
# 예측한 값(test_prediction)을 Submission 데이터브레임의 'Survived' 칼럼에 넣어줌
submission['Survived'] = test_prediction

In [82]:
submission.head(10)

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
5,897,0
6,898,0
7,899,0
8,900,1
9,901,0


In [287]:
test_prediction

array([0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0,
       1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1,
       1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1,
       0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [83]:
# 제출할 파일 저장하기 (저장하는 파일명은 본인이 원하는 대로)
submission.to_csv('./titanic_rfmodel_0201.csv', index=False)

In [84]:
# 현재 폴더에 보면 titanic_rfmodel_0421.csv 파일이 생성된 것을 확인할 수 있음.
# 이 파일을 https://www.kaggle.com/competitions/titanic/submissions에 제출하면 됨.
# 제출후 잠시 기다리면 https://www.kaggle.com/competitions/titanic/leaderboard에 점수와 순위가 나옴.

# 7. 과제

* 지금까지 배운 지식을 바탕으로 모델을 수정하면서 점수를 높여보기 바람.

* 맨 땅에 헤딩하면 어려우니 https://www.kaggle.com/competitions/titanic/code에서 다른 사람들이 올린 코드를 참고하기 바람. 
* https://www.kaggle.com/code/alexisbcook/titanic-tutorial
* https://www.kaggle.com/code/ash316/eda-to-prediction-dietanic

* 또는 구글에서 '타이타닉 머신러닝' 으로 검색하면 다른 사람들이 올린 코드를 참고할 수 있음.
* https://datainclude.me/posts/%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D%EC%9D%84_%EC%9D%B4%EC%9A%A9%ED%95%9C_%ED%83%80%EC%9D%B4%ED%83%80%EB%8B%89_%EC%83%9D%EC%A1%B4%EC%9E%90_%EC%98%88%EC%B8%A1/

# 8. 참고 (전체 코드)

In [149]:
'''
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# 1.데이터 불러오기
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

# 2.전처리
new_train = train[['Survived', 'Pclass', 'Sex', 'SibSp', 'Parch', 'Fare', 'Embarked']]
new_train.dropna(inplace=True)

new_train['Sex'] = new_train['Sex'].replace({'male': 0, 'female': 1})
new_train['Embarked'] = new_train['Embarked'].replace({'S': 0, 'C': 1, 'Q': 2})

# 3.학습
y = new_train['Survived']
X = new_train.drop('Survived', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2024)

rf_clf = RandomForestClassifier(random_state=2024)  
rf_clf.fit(X_train, y_train) 

# 4.평가
prediction = rf_clf.predict(X_test) 
rf_score = accuracy_score(y_test, prediction)
print('모델 정확도 :', rf_score)

# 5.예측
new_test = test[['Pclass', 'Sex', 'SibSp', 'Parch', 'Fare', 'Embarked']]
new_test['Sex'] = new_test['Sex'].replace({'male': 0, 'female': 1})
new_test['Embarked'] = new_test['Embarked'].replace({'S': 0, 'C': 1, 'Q': 2})
new_test['Fare'].fillna(new_test['Fare'].mean(), inplace=True)

test_prediction = rf_clf.predict(new_test)
print('예측값 :', test_prediction)

# 6.예측값 저장 및 제출
submmision = pd.read_csv('./data/gender_submission.csv')
submmision['Survived'] = test_prediction
submmision.to_csv('./titanic_rfmodel_0201.csv', index=False)
'''

모델 정확도 : 0.8033707865168539
예측값 : [0 1 0 0 1 0 0 0 1 0 0 0 1 0 1 1 0 0 0 1 1 0 1 0 1 0 1 0 1 0 0 0 1 0 1 0 0
 0 0 1 1 0 0 1 1 0 1 0 1 1 0 0 1 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 1 1 0 0 0
 1 0 0 1 0 1 1 0 0 0 0 0 1 0 1 1 0 0 1 0 1 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0
 1 1 0 1 0 0 1 1 1 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 1 1 0 0 1 0 0 1 0 0 1 1 1 1 0 0 1 0 0 1 0 0 0 0 0 0 1 1 1 1 1 0 1 1 0 1
 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0 1 0 1 0 0 0 0 1 1 0 1 0 1 0 1 0
 1 0 1 0 0 1 0 0 0 1 0 0 1 0 1 0 1 1 1 1 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 0 0 1 0 1 1 0 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 1 0
 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 1 1 0 0 0 1 0 1 0 0 0 0 1 1 0 1 0 0 0 1 1
 0 1 0 0 1 1 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 1 0 0 1 0 1 0 0 1 0 1 0 0 1 0
 0 1 0 1 1 0 0 1 0 0 1]
